# Browser Automation Lecture Notes
Date: 2023-07-18

We'll use this notebook to work through some examples and showcase some essential functions in Selenium.

Rather than basic Selenium, we'll use Selenium Wire, which can be used to intercept API calls/network requests.

In [76]:
!pip install selenium selenium-wire chromedriver-binary-auto

In [78]:
import os
import random
import time

from seleniumwire import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import (
    MoveTargetOutOfBoundsException,
    TimeoutException,
    WebDriverException,
)

import chromedriver_binary

In [79]:
os.makedirs('data/', exist_ok=True)

In [84]:
def open_browser():
    """
    Opens a new automated browser window with all tell-tales of automated browser disabled
    """
    options = webdriver.ChromeOptions()
    options.add_argument("start-maximized")
    
    # remove all signs of this being an automated browser
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)

    # open the browser with the new options
    driver = webdriver.Chrome(options=options)
    return driver

In [85]:
driver = open_browser()

In [86]:
url = 'https://amazon.com'
driver.get(url)

## XPATH
You learned about beautifulSoup to work with HTML...

Xpath is another way to donavigating hierarchical structures of HTML and SVG

It's fast, versatile, and can be used in the developer console and in any computing language.

## Using XPATH
You can test xpaths in the devloper tools under the `console` tab using the function `$x()`. Read about that function [here](https://developer.chrome.com/docs/devtools/console/utilities/#xpath-function).


You can use Selenium's `find_element` [function](https://selenium-python.readthedocs.io/locating-elements.html?highlight=find_element#locating-elements) to find the search box on Amazon site. It'll return the first match to whatever criteria you use.

You can use whichever `By` [option](https://selenium-python.readthedocs.io/api.html?highlight=By#locate-elements-by) you feel most comfortable. This includes Xpath, but also any kind of CSS selector.

Example of an xpath!

.//input[@aria-label="Search Amazon"]

In [89]:
# here is another way to do the same thing. Note that it returns multiple results
search_box = driver.find_elements(
    By.CLASS_NAME, 
    'nav-input'
)
search_box

[<selenium.webdriver.remote.webelement.WebElement (session="978e888df1c8ef34b806ae221a1ec784", element="71C0C108C87DE4A043A13AB5DDF3723D_element_224")>,
 <selenium.webdriver.remote.webelement.WebElement (session="978e888df1c8ef34b806ae221a1ec784", element="71C0C108C87DE4A043A13AB5DDF3723D_element_410")>]

In [94]:
search_box = driver.find_element(
    By.XPATH, 
    './/input[@aria-label="Search Amazon"]'
)
search_box

<selenium.webdriver.remote.webelement.WebElement (session="978e888df1c8ef34b806ae221a1ec784", element="93A2129CC1545A79D46F8A8DD5F1C215_element_499")>

In [97]:
search_term = 'womens shirt'
search_box.send_keys(search_term)

In [10]:
def press_enter(driver):
    """
    Sends the ENTER to a webdriver instance.
    """
    actions = ActionChains(driver)
    actions.send_keys(Keys.ENTER)
    actions.perform()

In [100]:
press_enter(driver)

### Parse the products

For each product, let's print the brand name:

Notice we're using `find_elements` (plural), this will return a list, rather than the first element.

In [101]:
product_tiles = driver.find_elements(
    By.XPATH, 
    './/div[contains(@cel_widget_id, "MAIN-SEARCH_RESULTS-")]' # the contains syntac allows a substring match
)
len(product_tiles)

60

In [103]:
product_tiles[0].text

"Best Seller\n+57 colors/patterns\nFeatured from Amazon brands\nAmazon Essentials\nWomen's Classic-Fit Short-Sleeve V-Neck T-Shirt, (Available in Plus Size), Multipacks\n42,894\n2K+ bought in past month\n$21\n20\nFREE delivery Sun, Jul 23 on $25 of items shipped by Amazon\nOr fastest delivery Fri, Jul 21\nPrime Try Before You Buy\nAmazon brand"

Let's iterate through each product and print the brand name, which is saved as the only header (h2) in the element:

In [104]:
for product in product_tiles:
    print(product.find_element(
        By.TAG_NAME, 'h2'
    ).text)

Amazon Essentials
Ladiyo
Amazon Essentials
Hanes
Amazon Essentials
Blooming Jelly
Lunivop
Hotouch
ayreus
Hanes
Amazon Essentials
LIYOHON
LIOFOER
ORANDESIGNE
NANYUAYA
Amazon Essentials
Amazon Essentials
PIIRESO
ayreus
LAISHEN
APOFER
Cicy Bell
Hanes
GAP
Amazon Essentials
Adibosy
Bestbee
SimpleFun
IWOLLENCE
Hanes
Amazon Essentials
Amazon Essentials
Dokotoo
Hanes
SheIn
MIHOLL
Micoson
Amazon Essentials
Amazon Essentials
The Drop
Anymiss
Hanes
Bealatt
Under Armour
PRETTYGARDEN
Carhartt
Hanes
Amazon Essentials
Amazon Essentials
Champion
ETCYY
ETCYY
DittyandVibe
Amazon Essentials
Blooming Jelly
Amazon Essentials
Hanes
Hanes
Under Armour
Nautica


Although we did this all using Selenium, it's better to save the page source and then parse the saved results in BeautifulSoup, lxml, or whatever parsing software you prefer.

### Annotate the elements we find
Let's find all the ads, and highlight them red on the page.

You can "inject" attributes into elements, including style attributes.

In [106]:
def stain(driver, elem, color = 'red'):
    """
    Injects a style attribute to stain `elem` the `color` red.
    """
    style = f"background-color: {color} !important; "\
                    "transition: all 0.5s linear;"
    driver.execute_script(
            f"arguments[0].setAttribute('style','{style}')", elem
    )

In [107]:
ads = driver.find_elements(
    By.XPATH, 
    # you can use XPATH to specify the attributes of the children of the node you want...
    './/div[@data-asin and .//a[@aria-label="View Sponsored information or leave ad feedback"]]'
)

In [108]:
len(ads)

13

In [109]:
for elem in ads:
    stain(driver, elem)

### Get height of document

In [17]:
import pandas as pd

In [18]:
height = driver.execute_script("return document.body.scrollHeight")

In [19]:
height

15187

Get the coorindates and size of each element using the `rect` function.

In [20]:
ad_metadata = []
for elem in ads:
    if elem.is_displayed(): # use this function to only analyze visable elements
        ad_metadata.append(elem.rect)

In [21]:
df = pd.DataFrame(ad_metadata)

In [22]:
df['how_far_down'] = df['y'] / height

In [23]:
df.how_far_down.value_counts()

0.033690    3
0.241418    2
0.286289    2
0.329579    2
0.196548    1
0.122904    1
Name: how_far_down, dtype: int64

### Save receipts

In [105]:
# how to save what the emulator sees
source = driver.page_source
with open('data/amazon_selenium_women_sshirts.html', 'w') as f:
    f.write(source)

In [25]:
# just what's visible
driver.save_screenshot('data/amazon_selenium_test.png')

True

There's are ways to do a full screen screenshot, but none of my function seem too work. Can you take a full-screenshot?

### Parsing the results however you like
For me it means using lxml, but you can do this same thing in BeautifulSoup, and I encourage you do so...

In [41]:
from lxml import etree

In [48]:
dom = etree.HTML(open('data/amazon_selenium_test.html').read())

In [67]:
result

<Element div at 0x7f77f38bb408>

In [73]:
product_metadata = []
for result in dom.xpath('.//div[contains(@cel_widget_id, "MAIN-SEARCH_RESULTS")]'):
    # this is where you can parse as many fields as you like.
    brand, product_name = result.xpath('.//h2//text()')[:2]
    product_metadata.append({
        'brand': brand,
        'product_name': product_name
    })

In [75]:
pd.DataFrame(product_metadata)

,brand,product_name
0,Amazon Essentials,Men's Short-Sleeve Chambray Shirt
1,Eycuro,Mens Henley T-Shirts Summer Short Sleeve 3/4 B...
2,JMIERR,Men's Linen Shirts Casual Long Sleeve Button-D...
3,COOFANDY,Men's Casual Short Sleeve Button Down Shirt Te...
4,Gildan,"Men's Crew T-Shirts, Multipack, Style G1100"
5,Carhartt,Men's Loose Fit Heavyweight Short-Sleeve Pocke...
6,Gildan,"Men's V-neck T-shirts, Multipack, Style G1103"
7,Hanes,"Mens Beefyt T-Shirt, Classic Heavyweight Cotto..."
8,Amazon Essentials,"Men's Short-Sleeve Crewneck T-Shirt, Pack of 2"
9,Nautica,Men's Solid Crew Neck Short-Sleeve Pocket T-Shirt


### SheWin, She Spin... or rotate
1. Find all products
2. filter to those with brand == SHEWIN
3. Find the image
    - Inject Javascript to make it spin.

In [129]:
def spin(driver, elem):
    """
    Injects a style attribute to rotate `elem` 180 degrees.
    """
    style = f"transform: rotate(180deg) !important; "
    driver.execute_script(
            f"arguments[0].setAttribute('style','{style}')", elem
    )

In [117]:
for elem in ads:
    spin(driver, elem)

In [123]:
brands_to_spin = [
    'SHEWIN',
    'Blooming Jelly'
]

In [125]:
product_tiles = driver.find_elements(
    By.XPATH, 
    './/div[contains(@cel_widget_id, "MAIN-SEARCH_RESULTS-")]' # the contains syntac allows a substring match
)
len(product_tiles)

60

## Here's the bountry
Ultimately, I want every product image from specific brands (of y0ur choosing) to rotate continuously.
You need a full screenshot or a video is fine, too.

Also, make sure to save the results before you parse them.

In [131]:
for product in product_tiles:
    # get the brand name...
    brand_name = product.find_element(
        By.TAG_NAME, 'h2'
    ).text
    
    # check if brand is in list
    if brand_name in brands_to_spin:
        print(product.text)
        # find the image TK
        spin(driver, product)

+22
Blooming Jelly
Womens White Blouse V Neck Ruffle Sleeve Flowy Shirts Dressy Casual Cute Summer Tops
4,633
1K+ bought in past month
$31
99 List:
$34.99
FREE delivery Sun, Jul 23
Or fastest delivery Fri, Jul 21
+7
Blooming Jelly
Women's Dressy Casual Tops Business Work Blouses White Button Down Shirts Cap Sleeve V Neck Tshirt
130
100+ bought in past month
$28
99
FREE delivery Sun, Jul 23
Or fastest delivery Fri, Jul 21
